## Goal

1. 判断每个自然年度的实际收益率
2. 判断每支基金哪个定投周期（投资策略）收益率最高
3. 判断是否买入新基金，整个基金组合的调整

In [1]:
import pandas as pd
import numpy as np
import datetime as dt
import warnings
warnings.filterwarnings("ignore")

In [2]:
df=pd.read_excel("001508.xlsx",sheet_name="2019")
total=5000

In [3]:
df.head()

,日期,单位净值,累计净值,净值增长率
0,2019-01-02,1.282,1.282,-0.0169
1,2019-01-03,1.277,1.277,-0.0039
2,2019-01-04,1.295,1.295,0.0141
3,2019-01-07,1.305,1.305,0.0077
4,2019-01-08,1.301,1.301,-0.0031


In [4]:
df["Week"]=df["日期"].dt.week
df["S/D"] = np.where(df["Week"]%2==1, "S", "D")
df["星期"]=df["日期"].dt.dayofweek+1
df["Day"]=df["日期"].dt.day
df.head(10)

,日期,单位净值,累计净值,净值增长率,Week,S/D,星期,Day
0,2019-01-02,1.282,1.282,-0.0169,1,S,3,2
1,2019-01-03,1.277,1.277,-0.0039,1,S,4,3
2,2019-01-04,1.295,1.295,0.0141,1,S,5,4
3,2019-01-07,1.305,1.305,0.0077,2,D,1,7
4,2019-01-08,1.301,1.301,-0.0031,2,D,2,8
5,2019-01-09,1.304,1.304,0.0023,2,D,3,9
6,2019-01-10,1.299,1.299,-0.0038,2,D,4,10
7,2019-01-11,1.300,1.300,0.0008,2,D,5,11
8,2019-01-14,1.293,1.293,-0.0054,3,S,1,14
9,2019-01-15,1.306,1.306,0.0101,3,S,2,15


In [5]:
def calyrr():
    df["累计资金"]=df["投入资金"].cumsum()
    df["购买份额"]=df["投入资金"]/df["单位净值"]
    df["累计份额"]=df["购买份额"].cumsum()
    df["当前资产"]=df["累计份额"]*df["单位净值"]
    df["YRR"]=(df["当前资产"]-df["累计资金"])/df["累计资金"]
def fundin(index):
    cnt=index.shape[0]
    df["投入资金"]=0
    df.loc[index,"投入资金"]=total/cnt

In [6]:
def yrr():
    #年定投
    yrr = pd.DataFrame(columns=['yEnd','max','min','maxDate','minDate'])
    df["投入资金"]=0
    df["投入资金"].iloc[0]=total
    df["投入资金"].iloc[1:]=0
    calyrr()
    newrow = pd.Series(data={'yEnd':"{:.2%}".format(df.iloc[-1]["YRR"]), 
                             'max':"{:.2%}".format(df["YRR"].max()), 
                             'min':"{:.2%}".format(df["YRR"].min()),
                             'maxDate':df["日期"][df["YRR"].index.max()],
                             'minDate':df["日期"][df["YRR"].index.min()]}, 
                       name='年初定投')
    yrrlist=yrr.append(newrow,ignore_index=False)
    
    #日定投
    cnt=df.shape[0]
    df["投入资金"]=0
    df["投入资金"]=total/cnt
    calyrr()
    newrow = pd.Series(data={'yEnd':"{:.2%}".format(df.iloc[-1]["YRR"]), 
                             'max':"{:.2%}".format(df["YRR"].max()), 
                             'min':"{:.2%}".format(df["YRR"].min()),
                             'maxDate':df["日期"][df["YRR"].index.max()],
                             'minDate':df["日期"][df["YRR"].index.min()]}, 
                       name='每日定投')
    yrrlist=yrrlist.append(newrow,ignore_index=False)
    
    #每周定投
    m=1
    for m in range(1,6):
        index=df[(df["星期"]==m)].index
        fundin(index)
        calyrr()
        newrow = pd.Series(data={'yEnd':"{:.2%}".format(df.iloc[-1]["YRR"]), 
                             'max':"{:.2%}".format(df["YRR"].max()), 
                             'min':"{:.2%}".format(df["YRR"].min()),
                             'maxDate':df["日期"][df["YRR"].index.max()],
                             'minDate':df["日期"][df["YRR"].index.min()]}, 
                       name=f"每周{m}定投")
        yrrlist=yrrlist.append(newrow,ignore_index=False)
        m=m+1
        
    m=1
    for m in range(1,6):
        index=df[(df["S/D"]=="S")&(df["星期"]==m)].index #奇数周
        fundin(index)
        calyrr()
        newrow = pd.Series(data={'yEnd':"{:.2%}".format(df.iloc[-1]["YRR"]), 
                             'max':"{:.2%}".format(df["YRR"].max()), 
                             'min':"{:.2%}".format(df["YRR"].min()),
                             'maxDate':df["日期"][df["YRR"].index.max()],
                             'minDate':df["日期"][df["YRR"].index.min()]}, 
                       name=f"每单周{m}定投")
        yrrlist=yrrlist.append(newrow,ignore_index=False)
        m=m+1
        
    m=1
    for m in range(1,6):
        index=df[(df["S/D"]=="D")&(df["星期"]==m)].index #偶数周
        fundin(index)
        calyrr()
        newrow = pd.Series(data={'yEnd':"{:.2%}".format(df.iloc[-1]["YRR"]), 
                             'max':"{:.2%}".format(df["YRR"].max()), 
                             'min':"{:.2%}".format(df["YRR"].min()),
                             'maxDate':df["日期"][df["YRR"].index.max()],
                             'minDate':df["日期"][df["YRR"].index.min()]}, 
                       name=f"每双周{m}定投")
        yrrlist=yrrlist.append(newrow,ignore_index=False)
        m=m+1
    
    #月定投 
    n=1
    for n in range(1,29):
        index=df[df["Day"]==n].index
        fundin(index)
        calyrr()
        newrow = pd.Series(data={'yEnd':"{:.2%}".format(df.iloc[-1]["YRR"]), 
                             'max':"{:.2%}".format(df["YRR"].max()), 
                             'min':"{:.2%}".format(df["YRR"].min()),
                             'maxDate':df["日期"][df["YRR"].index.max()],
                             'minDate':df["日期"][df["YRR"].index.min()]}, 
                       name=f"每月{n}定投")
        yrrlist=yrrlist.append(newrow,ignore_index=False)
        n=n+1
        
    return (yrrlist)

In [7]:
yrr=yrr()
yrr

,yEnd,max,min,maxDate,minDate
年初定投,74.26%,78.08%,-0.39%,2019-12-31,2019-01-02
每日定投,28.20%,32.23%,-0.20%,2019-12-31,2019-01-02
每周1定投,27.85%,31.79%,-0.46%,2019-12-31,2019-01-02
每周2定投,27.38%,31.32%,-0.61%,2019-12-31,2019-01-02
每周3定投,28.74%,32.85%,-0.39%,2019-12-31,2019-01-02
每周4定投,28.68%,32.78%,0.00%,2019-12-31,2019-01-02
每周5定投,28.34%,32.44%,-0.35%,2019-12-31,2019-01-02
每单周1定投,28.45%,32.45%,0.00%,2019-12-31,2019-01-02
每单周2定投,27.13%,31.39%,-0.26%,2019-12-31,2019-01-02
每单周3定投,30.10%,34.25%,-0.39%,2019-12-31,2019-01-02


In [8]:
yrr.to_excel("001508_2019.xlsx")